Here we use the Titanic Kaggle competition to test out our implementation of the decision tree. The script below trains a tree that achieves 78.9% of classification accuracy on the competition, which is a decent improvement on the gender-model (predicting that all men die and all women survive).

In this competition we try to build a model that can correctly predict which pasengers survived the Titanic accident. Accuracy is hence the metric we try to optimise on.

As a hyperparameter we need to chose the depth of the tree. We thus take 25% of the training data aside as validation set and use the precision on the validation set to set the depth of the tree. After we found this depth, we train the whole tree on the complete training data and see what the accuracy on the test data is (by submitting to kaggle).
Any further hyper parameter optimisation is beyond the scope of this example (e.g. minimum examples in each leaf).

This implementation of the decision tree is using Information gain as a splitting criterion. At each node, we check all features and chose the feature and split-value that maximise the information gain.
See, e.g.: http://chem-eng.utoronto.ca/~datamining/dmc/decision_tree.htm

In [1]:
import numpy as np
import random
from rforest import DecisionTree
import data_preparation as dataPrep

We set a couple of parameters and fix the random seeds for reproducability.

In [2]:
r_train = 0.75  # Fraction of data used as training data
N_ftr = 9  # number of features in the dataset
depths = range(2,10)  # list of different tree depths (need to find best hyperparameter here)
Nlabel = 2  # number of target values (dead and alive)

In [3]:
np.random.seed(1)
random.seed(1)

We load the data and split it into train and validation set. Our training data is a list here. The first list item contains training examples of people that died, the second list item of people that survived. This choice helps us in the tree to calculate the information gain, but is specific to this implementation and not compatible with libraries, such as sklearn.

In [4]:
data = dataPrep.titanic('train.csv', LABELED=True)
N0 = int(r_train * np.asarray(data[0]).shape[0])
N1 = int(r_train * np.asarray(data[1]).shape[0])


In [5]:
trainData  = [data[0][:N0, :], data[1][:N1, :]]
print np.asarray(trainData[0]).shape, np.asarray(trainData[1]).shape

(411, 10) (256, 10)


In [6]:
crossValData = [np.concatenate([data[0][N0 + 1:, :], data[1][N1 + 1:, :]])]
print np.asarray(crossValData).shape

(1, 222, 10)


We now define a helper function that calculates the error (misclassification rate) and the accuracy. It will evaluate each tree that we build.

In [7]:
def calculate_error(output):
    n_examples = output.shape[0]
    prediction = output[:, N_ftr]
    target = output[:, N_ftr + 1]
    misclass = np.where(target == prediction, 0, 1)
    error = np.mean(misclass)
    precision = 1. - error
    return error, precision

We use a basic loop to find the best tree-depth.
After the loop, we use the best depth and initialise a new tree. We then use the complete training data to train that tree.
We use that new tree to predict the test data and create a submit file for the Kaggle webpage.
We will find that the best depth is 4.
We will also see that the deeper the tree, the better the accuracy on the training data. This shows that our implementation is working as it can overfit the training data.

In [8]:
best_accuracy = 0.
best_depth = 0
for depth in depths:
    print depth
    RF = DecisionTree(depth, N_ftr, 2)
    RF.train([trainData])

    output = RF.predict_label([np.concatenate(trainData)])
    error, accuracy = calculate_error(output)
    print 'error and accuracy on training set: {}, {}'.format(
        round(error, 3), round(accuracy, 3))

    output = RF.predict_label(crossValData)
    error, accuracy = calculate_error(output)
    print 'error and accuracy on validation set: {}, {}\n'.format(
        round(error, 3), round(accuracy, 3))

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_depth = depth

print best_depth
trainData  = [data[0][:, :], data[1][:, :]]
RF = DecisionTree(best_depth, N_ftr, 2)
RF.train([trainData])

testData = dataPrep.titanic('test.csv', LABELED=False)
print np.asarray(testData).shape

output = RF.predict_label(testData)
output = zip(output[:, N_ftr], output[:, N_ftr + 1])
output.sort(key=lambda tup: tup[0])
pId, survival = zip(*output)

2
error and accuracy on training set: 0.211, 0.789
error and accuracy on validation set: 0.22, 0.78

3
error and accuracy on training set: 0.208, 0.792
error and accuracy on validation set: 0.215, 0.785

4
error and accuracy on training set: 0.171, 0.829
error and accuracy on validation set: 0.161, 0.839

5
error and accuracy on training set: 0.166, 0.834
error and accuracy on validation set: 0.175, 0.825

6
error and accuracy on training set: 0.148, 0.852
error and accuracy on validation set: 0.175, 0.825

7
error and accuracy on training set: 0.132, 0.868
error and accuracy on validation set: 0.175, 0.825

8
error and accuracy on training set: 0.108, 0.892
error and accuracy on validation set: 0.179, 0.821

9
error and accuracy on training set: 0.097, 0.903
error and accuracy on validation set: 0.175, 0.825

4
(1, 418, 10)


In [9]:
with open('submit.txt', 'w') as f:
    f.write('PassengerId,Survived\n')
for ii in range(len(pId)):
    if ii == 0:
        continue
    with open('submit.txt', 'a') as f:
        f.write(str(int(pId[ii])) + ',' + str(int(survival[ii])) + '\n')

When we submit this file to Kaggle, we obtain an accuracy on the test set of 78.9%.
This is an imporvement on the gender model (which predicts that all men die and all women survive) and shows that our implementation of a decision tree is working well.